In [ ]:
%pip install yfinance
%pip install matplotlib
%pip install pandas numpy
%pip install --upgrade sympy torch
%pip install torch torchvision torchaudio

# 3. Restart runtime again

# 4. Now import and run your code
import torch
print(torch.__version__)

: 

# N-BEATS + Topological Data Analysis

In [ ]:
"""
NBEATS + TDA Standalone Implementation
Train only the TDA-enhanced model and save results for comparison
Uses ripser for real persistent homology computation
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from scipy.spatial.distance import pdist, squareform

# Install: !pip install ripser

try:
    from ripser import ripser
    RIPSER_AVAILABLE = True
    print("Ripser library available")
except ImportError:
    RIPSER_AVAILABLE = False
    print("⚠️ Install ripser: pip install ripser")
    exit()

# ==================== TDA FROM SCRATCH ====================

def takens_embedding(time_series, embedding_dim=3, time_delay=1):
    """Create Takens embedding (phase space reconstruction)"""
    n = len(time_series)
    m = n - (embedding_dim - 1) * time_delay
    
    if m <= 0:
        raise ValueError("Time series too short for given embedding parameters")
    
    embedded = np.zeros((m, embedding_dim))
    for i in range(m):
        for j in range(embedding_dim):
            embedded[i, j] = time_series[i + j * time_delay]
    
    return embedded


def compute_persistence_diagrams(point_cloud, max_dimension=1):
    """Compute persistence diagrams using Vietoris-Rips complex"""
    if not RIPSER_AVAILABLE:
        raise ImportError("ripser not installed")
    
    from ripser import ripser as ripser_compute
    result = ripser_compute(point_cloud, maxdim=max_dimension, thresh=np.inf)
    return result['dgms']


def persistence_entropy(diagram):
    """Compute persistence entropy"""
    if len(diagram) == 0:
        return 0.0
    
    diagram = diagram[np.isfinite(diagram).all(axis=1)]
    if len(diagram) == 0:
        return 0.0
    
    lifetimes = diagram[:, 1] - diagram[:, 0]
    lifetimes = lifetimes[lifetimes > 0]
    
    if len(lifetimes) == 0:
        return 0.0
    
    lifetimes = lifetimes / np.sum(lifetimes)
    entropy = -np.sum(lifetimes * np.log(lifetimes + 1e-10))
    
    return float(entropy)


def max_persistence(diagram):
    """Maximum persistence (lifetime) of any feature"""
    if len(diagram) == 0:
        return 0.0
    
    diagram = diagram[np.isfinite(diagram).all(axis=1)]
    if len(diagram) == 0:
        return 0.0
    
    lifetimes = diagram[:, 1] - diagram[:, 0]
    return float(np.max(lifetimes))


def total_persistence(diagram):
    """Total persistence (sum of all lifetimes)"""
    if len(diagram) == 0:
        return 0.0
    
    diagram = diagram[np.isfinite(diagram).all(axis=1)]
    if len(diagram) == 0:
        return 0.0
    
    lifetimes = diagram[:, 1] - diagram[:, 0]
    return float(np.sum(lifetimes))


def number_of_features(diagram, threshold=0.0):
    """Count features with persistence above threshold"""
    if len(diagram) == 0:
        return 0
    
    diagram = diagram[np.isfinite(diagram).all(axis=1)]
    if len(diagram) == 0:
        return 0
    
    lifetimes = diagram[:, 1] - diagram[:, 0]
    return int(np.sum(lifetimes > threshold))


def betti_numbers(diagram, t):
    """Compute Betti number at filtration value t"""
    if len(diagram) == 0:
        return 0
    
    diagram = diagram[np.isfinite(diagram).all(axis=1)]
    alive = np.sum((diagram[:, 0] <= t) & (diagram[:, 1] > t))
    
    return int(alive)


class TDAFeatureExtractor:
    """Complete TDA feature extractor using ripser"""
    
    def __init__(self, embedding_dim=3, time_delay=1, max_homology_dim=1):
        self.embedding_dim = embedding_dim
        self.time_delay = time_delay
        self.max_homology_dim = max_homology_dim
        
    def extract_features(self, time_series):
        """
        Extract 12 TDA features from time series:
        H0: entropy, max_pers, total_pers, n_features, betti, amplitude
        H1: entropy, max_pers, total_pers, n_features, betti, amplitude
        """
        try:
            # Takens embedding
            embedded = takens_embedding(time_series, self.embedding_dim, self.time_delay)
            
            # Compute persistence diagrams
            diagrams = compute_persistence_diagrams(embedded, max_dimension=self.max_homology_dim)
            
            # Extract features
            features = []
            
            for dim in range(self.max_homology_dim + 1):
                diagram = diagrams[dim]
                
                # 1. Entropy
                features.append(persistence_entropy(diagram))
                
                # 2. Max persistence
                max_pers = max_persistence(diagram)
                features.append(max_pers)
                
                # 3. Total persistence
                features.append(total_persistence(diagram))
                
                # 4. Number of significant features (threshold = 10% of max)
                threshold = 0.1 * max_pers if max_pers > 0 else 0.0
                features.append(float(number_of_features(diagram, threshold)))
                
                # 5. Betti number at midpoint
                if len(diagram) > 0:
                    diagram_finite = diagram[np.isfinite(diagram).all(axis=1)]
                    if len(diagram_finite) > 0:
                        midpoint = (np.min(diagram_finite[:, 0]) + np.max(diagram_finite[:, 1])) / 2
                        features.append(float(betti_numbers(diagram, midpoint)))
                    else:
                        features.append(0.0)
                else:
                    features.append(0.0)
                
                # 6. Amplitude (max death - min birth)
                if len(diagram) > 0:
                    diagram_finite = diagram[np.isfinite(diagram).all(axis=1)]
                    if len(diagram_finite) > 0:
                        amplitude = np.max(diagram_finite[:, 1]) - np.min(diagram_finite[:, 0])
                        features.append(float(amplitude))
                    else:
                        features.append(0.0)
                else:
                    features.append(0.0)
            
            return np.array(features[:12], dtype=np.float32)
            
        except Exception as e:
            print(f"Warning: TDA extraction failed: {e}")
            return np.zeros(12, dtype=np.float32)


# ==================== NBEATS COMPONENTS ====================

class NBeatsBlock(nn.Module):
    """NBEATS block with proper basis functions"""
    def __init__(self, input_size, theta_size, horizon, n_layers=4, layer_size=256):
        super().__init__()
        self.input_size = input_size
        self.theta_size = theta_size
        self.horizon = horizon
        
        layers = [nn.Linear(input_size, layer_size), nn.ReLU()]
        for _ in range(n_layers - 1):
            layers.extend([nn.Linear(layer_size, layer_size), nn.ReLU()])
        
        self.fc_stack = nn.Sequential(*layers)
        self.theta_b = nn.Linear(layer_size, theta_size, bias=False)
        self.theta_f = nn.Linear(layer_size, theta_size, bias=False)
        
        self.backcast_basis = nn.Linear(theta_size, input_size)
        self.forecast_basis = nn.Linear(theta_size, horizon)
        
    def forward(self, x):
        h = self.fc_stack(x)
        theta_b = self.theta_b(h)
        theta_f = self.theta_f(h)
        
        backcast = self.backcast_basis(theta_b)
        forecast = self.forecast_basis(theta_f)
        
        return backcast, forecast


class NBeatsStack(nn.Module):
    """Stack of NBEATS blocks"""
    def __init__(self, input_size, horizon, n_blocks=3, n_layers=4, layer_size=256):
        super().__init__()
        theta_size = max(input_size // 4, horizon // 4)
        
        self.blocks = nn.ModuleList([
            NBeatsBlock(input_size, theta_size, horizon, n_layers, layer_size)
            for _ in range(n_blocks)
        ])
        
    def forward(self, x):
        residual = x
        forecast = torch.zeros(x.size(0), self.blocks[0].horizon, device=x.device)
        
        for block in self.blocks:
            backcast, block_forecast = block(residual)
            residual = residual - backcast
            forecast = forecast + block_forecast
            
        return forecast, residual


class NBEATS_TDA(nn.Module):
    """NBEATS enhanced with TDA features"""
    def __init__(self, input_size, horizon, tda_feature_size=12, 
                 n_stacks=2, n_blocks=2, n_layers=4, layer_size=128):
        super().__init__()
        self.input_size = input_size
        self.horizon = horizon
        
        # NBEATS backbone
        self.stacks = nn.ModuleList([
            NBeatsStack(input_size, horizon, n_blocks, n_layers, layer_size)
            for _ in range(n_stacks)
        ])
        
        # TDA feature processor
        self.tda_processor = nn.Sequential(
            nn.Linear(tda_feature_size, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, horizon)
        )
        
        # Attention fusion
        self.attention = nn.Sequential(
            nn.Linear(horizon * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.Softmax(dim=1)
        )
        
        self.output_projection = nn.Linear(horizon, horizon)
        
    def forward(self, x, tda_features):
        # NBEATS forecast
        residual = x
        nbeats_forecast = torch.zeros(x.size(0), self.horizon, device=x.device)
        
        for stack in self.stacks:
            stack_forecast, residual = stack(residual)
            nbeats_forecast = nbeats_forecast + stack_forecast
        
        # TDA forecast
        tda_forecast = self.tda_processor(tda_features)
        
        # Attention fusion
        combined = torch.cat([nbeats_forecast, tda_forecast], dim=1)
        attention_weights = self.attention(combined)
        
        weighted_nbeats = nbeats_forecast * attention_weights[:, 0:1]
        weighted_tda = tda_forecast * attention_weights[:, 1:2]
        
        fused = weighted_nbeats + weighted_tda
        final_forecast = self.output_projection(fused)
        
        return final_forecast


# ==================== DATASET ====================

class TDADataset(Dataset):
    """Dataset with TDA features"""
    def __init__(self, data, input_size, horizon, tda_extractor):
        self.data = data
        self.input_size = input_size
        self.horizon = horizon
        self.tda_extractor = tda_extractor
        
        print("Pre-computing TDA features...")
        self.tda_features = self._precompute_tda()
        print(f"TDA features computed: shape {self.tda_features.shape}")
        
    def _precompute_tda(self):
        features_list = []
        total = len(self)
        
        for i in range(total):
            if (i + 1) % 100 == 0:
                print(f"  {i+1}/{total}...")
            
            window = self.data[i:i + self.input_size]
            features = self.tda_extractor.extract_features(window)
            features_list.append(features)
        
        return torch.FloatTensor(np.array(features_list))
    
    def __len__(self):
        return len(self.data) - self.input_size - self.horizon + 1
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.data[idx:idx + self.input_size])
        y = torch.FloatTensor(self.data[idx + self.input_size:idx + self.input_size + self.horizon])
        tda_feat = self.tda_features[idx]
        return x, y, tda_feat


# ==================== TRAINING ====================

class EarlyStopping:
    def __init__(self, patience=15, min_delta=1e-5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


def train_model(model, train_loader, val_loader, epochs, lr, device):
    """Train NBEATS+TDA model"""
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    early_stopping = EarlyStopping(patience=15)
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        # Training
        model.train()
        total_loss = 0
        for x, y, tda_feat in train_loader:
            x, y, tda_feat = x.to(device), y.to(device), tda_feat.to(device)
            
            optimizer.zero_grad()
            output = model(x, tda_feat)
            loss = criterion(output, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
        
        train_loss = total_loss / len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y, tda_feat in val_loader:
                x, y, tda_feat = x.to(device), y.to(device), tda_feat.to(device)
                output = model(x, tda_feat)
                loss = criterion(output, y)
                val_loss += loss.item()
        
        val_loss = val_loss / len(val_loader)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), '/tmp/best_nbeats_tda.pt')
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - Train: {train_loss:.6f}, Val: {val_loss:.6f}")
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"⚠️  Early stopping at epoch {epoch+1}")
            break
    
    # Load best model
    model.load_state_dict(torch.load('/tmp/best_nbeats_tda.pt'))
    print("Training complete! Loaded best model.")
    
    return model, train_losses, val_losses


def evaluate_model(model, test_loader, device):
    """Evaluate and get predictions"""
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for x, y, tda_feat in test_loader:
            x, tda_feat = x.to(device), tda_feat.to(device)
            output = model(x, tda_feat)
            all_preds.append(output.cpu().numpy())
            all_targets.append(y.numpy())
    
    predictions = np.concatenate(all_preds, axis=0)
    targets = np.concatenate(all_targets, axis=0)
    
    return predictions, targets


def compute_metrics(y_true, y_pred):
    """Compute MAE, RMSE, MAPE"""
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}


# ==================== MAIN ====================

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🖥️  Using device: {device}\n")
    
    # Download data
    ticker = 'GOOG'
    print(f"📊 Downloading {ticker} stock data...")
    data = yf.download(ticker, start='2015-01-01', end='2023-12-31', auto_adjust=True)
    
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    
    prices = data['Close'].values
    mean, std = prices.mean(), prices.std()
    prices_normalized = (prices - mean) / std
    
    # Split data
    train_size = int(0.7 * len(prices_normalized))
    val_size = int(0.15 * len(prices_normalized))
    
    train_data = prices_normalized[:train_size]
    val_data = prices_normalized[train_size:train_size + val_size]
    test_data = prices_normalized[train_size + val_size:]
    
    print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}\n")
    
    # Hyperparameters
    INPUT_SIZE = 60
    HORIZON = 20
    BATCH_SIZE = 32
    EPOCHS = 100
    LR = 1e-3
    
    # Create TDA extractor (using ripser-based implementation)
    print("Initializing TDA Feature Extractor with Ripser...")
    tda_extractor = TDAFeatureExtractor(embedding_dim=3, time_delay=1, max_homology_dim=1)
    print("Using real persistent homology computation\n")
    
    # Create datasets
    train_dataset = TDADataset(train_data, INPUT_SIZE, HORIZON, tda_extractor)
    val_dataset = TDADataset(val_data, INPUT_SIZE, HORIZON, tda_extractor)
    test_dataset = TDADataset(test_data, INPUT_SIZE, HORIZON, tda_extractor)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    print(f"\nTraining NBEATS+TDA Model")
    print("=" * 60)
    
    # Create model
    model = NBEATS_TDA(INPUT_SIZE, HORIZON, tda_feature_size=12, 
                       n_stacks=2, n_blocks=2, layer_size=128)
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}\n")
    
    # Train
    model, train_losses, val_losses = train_model(model, train_loader, val_loader, 
                                                    EPOCHS, LR, device)
    
    # Evaluate
    print("\nEvaluating on test set...")
    predictions, targets = evaluate_model(model, test_loader, device)
    
    # Denormalize
    predictions_denorm = predictions * std + mean
    targets_denorm = targets * std + mean
    
    # Compute metrics
    metrics = compute_metrics(targets_denorm, predictions_denorm)
    
    print("\n" + "=" * 60)
    print("NBEATS+TDA TEST RESULTS")
    print("=" * 60)
    print(f"  MAE:  ${metrics['MAE']:.2f}")
    print(f"  RMSE: ${metrics['RMSE']:.2f}")
    print(f"  MAPE: {metrics['MAPE']:.2f}%")
    print("=" * 60)
    
    # Save results for comparison
    results = {
        'model': 'NBEATS+TDA',
        'ticker': ticker,
        'input_size': INPUT_SIZE,
        'horizon': HORIZON,
        'test_samples': len(test_dataset),
        'metrics': {
            'MAE': float(metrics['MAE']),
            'RMSE': float(metrics['RMSE']),
            'MAPE': float(metrics['MAPE'])
        },
        'hyperparameters': {
            'n_stacks': 2,
            'n_blocks': 2,
            'layer_size': 128,
            'tda_features': 12,
            'embedding_dim': 3
        }
    }
    
    # Save to JSON
    with open('/tmp/nbeats_tda_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print("\nResults saved to: /tmp/nbeats_tda_results.json")
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Training curves
    axes[0, 0].plot(train_losses, label='Train Loss', alpha=0.8)
    axes[0, 0].plot(val_losses, label='Val Loss', alpha=0.8)
    axes[0, 0].set_title('Training Curves', fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('MSE Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_yscale('log')
    
    # Predictions vs Actual - FIXED: Show first time step only
    sample_idx = slice(0, 200)
    # Take only the first forecast step from each prediction window
    first_step_preds = predictions_denorm[sample_idx, 0]  # Shape: (200,)
    first_step_targets = targets_denorm[sample_idx, 0]    # Shape: (200,)
    
    axes[0, 1].plot(first_step_preds, label='Predicted (1-step)', linestyle='--', alpha=0.7, linewidth=2)
    axes[0, 1].plot(first_step_targets, label='Actual', linewidth=2, alpha=0.7)
    axes[0, 1].set_title('Predictions vs Actual (First Step of Each Window)', fontweight='bold')
    axes[0, 1].set_xlabel('Sample')
    axes[0, 1].set_ylabel('Price ($)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Error distribution - FIXED: Flatten properly
    errors = np.abs(targets_denorm - predictions_denorm)
    errors_flat = errors.flatten()
    
    axes[1, 0].hist(errors_flat, bins=50, alpha=0.7, edgecolor='black')
    axes[1, 0].axvline(np.mean(errors_flat), color='red', linestyle='--', linewidth=2, 
                       label=f'Mean: ${np.mean(errors_flat):.2f}')
    axes[1, 0].set_xlabel('Absolute Error ($)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Error Distribution (All Forecast Steps)', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Scatter: Actual vs Predicted - FIXED: Flatten properly
    axes[1, 1].scatter(targets_denorm.flatten(), predictions_denorm.flatten(), 
                      alpha=0.3, s=10, c=errors_flat, cmap='RdYlGn_r')
    min_val = min(targets_denorm.min(), predictions_denorm.min())
    max_val = max(targets_denorm.max(), predictions_denorm.max())
    axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect')
    axes[1, 1].set_xlabel('Actual Price ($)')
    axes[1, 1].set_ylabel('Predicted Price ($)')
    axes[1, 1].set_title('Actual vs Predicted (All Steps, Color=Error)', fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
    cbar.set_label('Error ($)', rotation=270, labelpad=15)
    
    plt.tight_layout()
    plt.savefig('/tmp/nbeats_tda_results.png', dpi=150, bbox_inches='tight')
    print("Plots saved to: /tmp/nbeats_tda_results.png")
    plt.show()
    
    print("\nNBEATS+TDA training complete!")
    print("\nTo compare with vanilla NBEATS:")
    print("   1. Note these metrics")
    print("   2. Run vanilla NBEATS separately")
    print("   3. Calculate improvement percentages")
    
    return model, metrics, results

if __name__ == "__main__":
    model, metrics, results = main()